## Work in progress

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import logging
import os

import seaborn as sns
import yaml
from dotenv import load_dotenv
from ultralytics import YOLO

logger = logging.getLogger()
logger.setLevel(logging.INFO)

load_dotenv("../.envrc")

ROOT_DIR = os.getenv("ROOT_DIR")
PARAMS = os.getenv("PARAMS")
os.chdir(ROOT_DIR)

sns.set_style("darkgrid")

## Export the model in ONNX format

In [ ]:
with open(PARAMS) as conf_file:
    config = yaml.safe_load(conf_file)

model_path = config["evaluate"]["best_model"]

model = YOLO(model_path)
model.export(format="onnx", imgsz=(256, 306), dynamic=False)
model.export(format="onnx", imgsz=306, dynamic=False)

In [ ]:
import cv2
import numpy as np
import onnxruntime as ort

# from ultralytics import non_max_suppression, xyxy2xywh
# from IPython.display import Image, display
# display(Image(filename='image.png'))

onnx_model = ort.InferenceSession("models/best_model.onnx")
inputs = onnx_model.get_inputs()
input = inputs[0]

print("Name:", input.name)
print("Type:", input.type)
print("Shape:", input.shape)

outputs = onnx_model.get_outputs()
output = outputs[0]
print("Name:", output.name)
print("Type:", output.type)
print("Shape:", output.shape)


def predict(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (320, 320), interpolation=cv2.INTER_LINEAR)
    image = np.expand_dims(image, axis=0).astype("float32") / 255.0
    image = np.transpose(image, (0, 3, 1, 2))
    logging.info(image.shape)

    outputs = onnx_model.run(None, {"images": image})

    print(outputs)

    output = outputs[0]
    print("Output shape:", output.shape)

    output = output.transpose()
    print("Output shape:", output.shape)

    row = output[0]
    print(row)
    x1, y1, x2, y2 = row[:4]
    print(x1, y1, x2, y2)

    # x1 = (xc - w/2) / 320 * 320
    # y1 = (yc - h/2) / 320 * 320
    # x2 = (xc + w/2) / 320 * 320
    # y2 = (yc + h/2) / 320 * 320

    # print(x1[0], y1[0], x2[0], y2[0])

    # img = cv2.imread("image.png")
    # cv2.rectangle(img, (11.995002, 1.6412115), (38.259865, 36.000008), (36, 255, 12), 1)
    # plt.imshow(img)
    # plt.axis(False)
    # plt.grid(False)
    # plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    # plt.show()


model = predict("data/plugs/test/images/random-plug.png")